In [10]:
from quspin.operators import quantum_operator
from quspin.basis import spin_basis_1d
from quspin.tools.evolution import evolve
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from hmftpy.plaquettes.square import plaq4
from hmftpy import do_hmft
from hmftpy.plot_spins import draw_tri_lattice, draw_cluster, plot_spins
from hmftpy.operators import inner_hamiltonian, periodic_hamiltonian

# Our model

Let's work with the square lattice. Our cluster will be 4 spins, with a symmetric tiling. For now, let's do antiferromagnetic Heisenberg interactions with an external (time-dependent) magnetic field, which starts switched off.

In [11]:
interactions = {'local': {'z': 0},
                'nearest': {'xx': 1, 'yy': 1, 'zz': 1}
               }
plaq = plaq4
basis = spin_basis_1d(4)
print(basis.Ns)
e, v, mf, cvg = do_hmft(plaq, interactions, basis)

H = inner_hamiltonian(plaq, interactions, basis)
ei, vi = H.eigh()

16


In [34]:
sx = np.zeros((basis.Ns, basis.Ns, 4), dtype=np.float64)
sy = np.zeros((basis.Ns, basis.Ns, 4), dtype=np.float64)
sz = np.zeros((basis.Ns, basis.Ns, 4), dtype=np.float64)
for i in range(basis.Ns):
    for j in range(i):
        for k in range(4):
            sx[i,j,k] = np.real(np.vdot(vi[:,i], basis.inplace_Op(vi[:,j], [['x', [k], 1]], np.complex128)))
            sx[j,i,k] = sx[i,j,k]
            sy[i,j,k] = np.real(np.vdot(vi[:,i], basis.inplace_Op(vi[:,j], [['y', [k], 1]], np.complex128)))
            sy[j,i,k] = sx[i,j,k]
            sz[i,j,k] = np.real(np.vdot(vi[:,i], basis.inplace_Op(vi[:,j], [['z', [k], 1]], np.complex128)))
            sz[j,i,k] = sx[i,j,k]

In [40]:
def get_c(v0, vs):
    cs = np.zeros(len(vs), dtype=np.complex128)
    for i, v in enumerate(vs):
        cs[i] = np.vdot(vs[i], v0)
    return cs
c0 = get_c(v, vi)
print(c0)

[-0.03318788-0.05648713j -0.02612999-0.06940526j -0.07358818-0.19546148j
  0.00507194+0.00636994j  0.07358818+0.19546148j  0.25586092+0.66785827j
  0.02220579+0.06608604j -0.01578743-0.03576495j -0.15229676-0.40452352j
  0.00480376-0.02491743j -0.11578283-0.33950386j -0.01105424+0.00469115j
 -0.09086621-0.23752831j  0.01699763+0.03760763j  0.00430132-0.01308337j
  0.02350441+0.04000548j]


# Time evolution equations
Our HMFT Hamiltonian is
\begin{align*}
H = -h(t) \sum_i \sigma_i^z
+ \sum_{i}\sum_{j\in \partial_i^\text{int}}\sigma_i \cdot \sigma_j 
+ \sum_{i} \sum_{j\in \partial^\text{ext}_i} \sigma_i \cdot  s_j(t).
\end{align*}
Then, we write our state as
\begin{equation}
|{\psi(t)\langle = \sum_n c_n(t) |n\rangle
\end{equation}
where $|n\rangle$ is an eigenstate of the Hamiltonian with $h=0$, with eigenvalue $\epsilon_n$. Then, the TDSE is

\begin{equation}
i \partial_t |\psi(t)\rangle =-h(t)\sum_i \sigma_i^z |\psi(t)\rangle
+ \sum_{i}\sum_{j\in \partial_i^\text{int}}\sigma_i \cdot \sigma_j |\psi(t)\rangle
+ \sum_{i} \sum_{j\in \partial^\text{ext}_i} \sigma_i \cdot  s_j(t)|\psi(t)\rangle.
\end{equation}
Now acting on this with the states $\langle n|$, we get the set of equations ($n = 1$ to 16)
\begin{align*}
i \dot c_n(t) 
=&-h(t)\sum_i \sum_m \langle n| \sigma_i^z |m\rangle c_m(t)
+ \sum_i \sum_{j\in \partial_i^\text{int}} \sum_{l,m} \sum_\mu \langle n|\sigma_i ^\mu|l \rangle  \langle l| \sigma_j^\mu | m \rangle c_m(t)
+ \sum_i \sum_{j\in \partial_i^\text{ext}} \sum_m\sum_\mu \langle n|\sigma_i^\mu|m \rangle  s_j^\mu(t) c_m(t)\\
=&\sum_i\sum_m \left(-h(t)\langle n| \sigma_i^z |m\rangle 
+ \sum_i \sum_{j\in \partial_i^\text{int}} \sum_{l,m} \sum_\mu \langle n|\sigma_i ^\mu|l \rangle  \langle l| \sigma_j^\mu | m \rangle 
+ \sum_i \sum_{j\in \partial_i^\text{ext}} \sum_m\sum_\mu \langle n|\sigma_i^\mu|m \rangle
\sum_{a,b}\langle a|\sigma_j^\mu|b\rangle
c_a^*(t)c_b(t)
\right)c_m(t)
\end{align*}

In [ ]:
def f(t, c):
    dc = np.zeros(basis.Ns)
    for n in range(basis.Ns):
        dc[n] = 1j*h(t)*np.sum(sz[n,:,:])
        for i in range(4):
            neighbors_in = plaq['inner']['nearest'][i]
            neighbors_out = plaq['outer']['nearest'][i]
            for l in range(basis.Ns):
                dc[n] += -1j*np.sum(sx[n,l,i]*sx[l,:,neighbors_in])
                dc[n] += -1j*np.sum(sy[n,l,i]*sy[l,:,neighbors_in])
                dc[n] += -1j*np.sum(sz[n,l,i]*sz[l,:,neighbors_in])
    return